In [ ]:
import yfinance as yf
import pywt
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from arch import arch_model
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# load the daily stock price of Tesla
ticker = 'TSLA'
#data = yf.download(ticker, start='2010-07-07', end='2023-02-17')
#data= data.to_csv("TSLA_close-price")
data = pd.read_csv("TSLA_close-price")
close_prices = data['Close']

# Calculate os returns 
data['Returns'] = data['Close'].pct_change().dropna()


#Calculate Volatility
window_size  = 5
realized_variance = data['Returns'].rolling(window=window_size).var()
data['Volatility'] =  np.sqrt(realized_variance)


In [ ]:
data['Date'] = pd.to_datetime(data['Date'])   
data.set_index('Date', inplace=True)
data.dropna(inplace =True)
data.head(3)

## TSLA Close price and plot for Corresponding volume

In [ ]:
plt.figure(figsize=(6,4))
plt.subplot(2, 1, 1)
plt.plot(data['Close'], label='Close Price')
plt.title('TSLA Close Price')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(data['Volume'], label='Volume', color='red')
plt.title('Rrading Volume')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
data=data[["Close","Returns","Volatility"]]
data.head(1)

## Plotting Returns and Volatility

In [ ]:
plt.figure(figsize=(6,4))
plt.subplot(2, 1, 1)
plt.plot(data['Returns'], label='Returns')
plt.title('Daily Returns')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(data['Volatility'], label='Volatility', color='red')
plt.title('Rolling Volatility')
plt.legend()

plt.tight_layout()
plt.show()

## STATIONARITY TEST

In [ ]:
from statsmodels.tsa.stattools import adfuller

#H0: It is non stationary
#H1: It is stationary

def test_stationarity(data, column):
    # Calculate rolling mean and standard deviation
    data['rollMean'] = data[column].rolling(window=12).mean()
    data['rollStd'] = data[column].rolling(window=12).std()
    
    # Perform the Augmented Dickey-Fuller test
    def adf_Test(series):
        result = adfuller(series, autolag='AIC')
        labels = ['ADF Test Statistic', 'P-value', '#Lags Used', 'Number of Observations Used']
        for value, label in zip(result[:4], labels):
            print(f'{label} : {value}')
        
        print('Critical Values:')
        for key, value in result[4].items():
            print(f'   {key} : {value}')

        if result[1] <= 0.05:
            print("Strong evidence against the null hypothesis (H0), reject the null hypothesis. HENCE>> The time series is stationary.")
        else:
            print("Weak evidence against the null hypothesis, time series has a unit root indicating it is non-stationary.")

    # Apply the ADF test
    adf_Test(data[column])

    # Plot the time series data, rolling mean, and rolling standard deviation
    plt.figure(figsize=(7, 3))
    sns.lineplot(data=data, x=data.index, y=column, label=column)
    sns.lineplot(data=data, x=data.index, y='rollMean', label='Rolling Mean')
    sns.lineplot(data=data, x=data.index, y='rollStd', label='Rolling Std')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
#Stationarity test for Returns
test_stationarity(data,"Returns")

In [ ]:
test_stationarity(data,"Volatility")

In [ ]:
data= data.dropna()
data.head()

# ----------------------MODELLING----------------------

In [ ]:
# Plot realized volatility  
plt.figure(figsize=(7, 4))
plt.plot(data["Volatility"][-42:], label='Actual Realized Volatility')
plt.title('Realized Volatility of Tesla Stock')
plt.xticks(rotation=45) 
plt.legend()

plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [ ]:
realized_volatility = data["Volatility"] .dropna() 
#Take all the data except for the last 21 days.
train_volatility = realized_volatility[:-21] 
#Test the model for the next 21 days.
test = realized_volatility[-21:] 

# ARIMA

In [ ]:
history = [x for x in train_volatility]
predictions = []
for t in range(len(test)):
    model = ARIMA(history, order=(4, 0, 4))
    model_fit = model.fit()
    yhat = model_fit.forecast()[0]
    predictions.append(yhat)
    history.append(test[t])

predictions = pd.Series(predictions, index=test.index)

In [ ]:

plt.figure(figsize=(5, 4))
plt.plot(realized_volatility[-42:] , label='Actual Realized Volatility') 
plt.plot(predictions.index, predictions, label='ARIMA Forecast', linestyle='--')
plt.xticks(rotation=45)  # Rotate x-axis labels by 90 degrees
plt.title('Realized Volatility and ARIMA Forecast')
plt.legend()
plt.show()

correlation_arima_x_original = realized_volatility[-21:].corr(predictions)
print(f'Correlation: {correlation_arima_x_original}')


slope, intercept = np.polyfit(realized_volatility[-21:], predictions, 1)
regression_line = slope * realized_volatility[-21:] + intercept

plt.figure(figsize=(5, 4))
plt.scatter(realized_volatility[-21:], predictions, c='blue', marker='o')
plt.plot(realized_volatility[-21:], regression_line, color='red', label='Linha de Regressão')
plt.xlabel('Original Series')
plt.ylabel('Prediction Series')
plt.title(f'Gráfico de Dispersão - Correlação: {correlation_arima_x_original:.2f}')
plt.grid(True)
plt.show()

def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(realized_volatility[-21:], predictions)
# Calcular o MAE
mae = mean_absolute_error(realized_volatility[-21:], predictions)
# Calcular o RMSE
rmse = np.sqrt(mean_squared_error(realized_volatility[-21:], predictions))
# Calcular o R^2
r2 = r2_score(realized_volatility[-21:], predictions)

# Exibir as métricas
print(f'MAPE: {mape:.2f}%')
print(f'MAE: {mae*100:.2f}')
print(f'RMSE: {rmse*100:.2f}')
print(f'R^2: {r2:.2f}%')

# GARCH

In [ ]:
# One-step-ahead forecast with GARCH
history_garch = [x for x in train_volatility.dropna()]
garch_predictions = []
for t in range(len(test)):
    model = arch_model(history_garch, mean='zero', vol='EGARCH', p=4, q=2, dist='Normal')
    model_fit = model.fit(disp="off")
    forecast = model_fit.forecast(horizon=1)
    yhat = np.sqrt(forecast.variance.values[-1, 0])
    garch_predictions.append(yhat)
    history_garch.append(test[t])

garch_predictions = pd.Series(garch_predictions, index=test.index)


In [ ]:
# 12. Plotting the results for GARCH model
plt.figure(figsize=(5,4))
plt.plot(realized_volatility[-42:], label='Actual Realized Volatility')
plt.plot(garch_predictions.index, garch_predictions, label='GARCH Forecast', linestyle='--')
plt.xticks(rotation=45)
plt.title('Realized Volatility and GARCH Forecast')
plt.legend()
plt.grid(True)
plt.show()


correlation_arima_x_original = realized_volatility[-21:].corr(garch_predictions)
print(f'Correlation: {correlation_arima_x_original}')


slope, intercept = np.polyfit(realized_volatility[-21:], garch_predictions, 1)
regression_line = slope * realized_volatility[-21:] + intercept

plt.figure(figsize=(5, 4))
plt.scatter(realized_volatility[-21:], garch_predictions, c='blue', marker='o')
plt.plot(realized_volatility[-21:], regression_line, color='red', label='Linha de Regressão')
plt.xlabel('Original Series')
plt.ylabel('Prediction Series')
plt.title(f'TSLA GARCH - Correlation: {correlation_arima_x_original:.2f}')
plt.grid(True)
plt.show()


# Error metrics
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
#Calculate MAPE
mape_GARCH = mean_absolute_percentage_error(realized_volatility[-21:], garch_predictions)
# Calculate MAE
mae_GARCH = mean_absolute_error(realized_volatility[-21:], garch_predictions)
# Calculate RMSE
rmse_GARCH = np.sqrt(mean_squared_error(realized_volatility[-21:], garch_predictions))
# Calculate  R^2
r2_GARCH = r2_score(realized_volatility[-21:], garch_predictions)

# Exibir as métricas
print(f'MAPE: {mape_GARCH:.2f}%')
print(f'MAE: {mae_GARCH*100:.2f}')
print(f'RMSE: {rmse_GARCH*100:.2f}')
print(f'R^2: {r2_GARCH:.2f}%')
 

  

 
 

# ARIMA-GARCH

In [ ]:
# Split the data into train and test sets
train_volatility = realized_volatility.dropna()[:-21]
test = realized_volatility.dropna()[-21:]

# Trainning ARIMA model
arima_model = ARIMA(train_volatility, order=(4, 0, 4)).fit()
arima_residuals = arima_model.resid

# Trainning GARCH model on ARIMA residuals
garch_model = arch_model(arima_residuals, mean='zero', vol='EGARCH', p=4, q=2)
garch_fit = garch_model.fit(disp="off")

# One-step-ahead forecast with ARIMA-GARCH
history_volatility = [x for x in train_volatility]
history_residuals = [x for x in arima_residuals]
combined_predictions_ResGARCH = []

for t in range(len(test)):
    # ARIMA Forecast
    arima_model = ARIMA(history_volatility, order=(4, 0, 4))
    arima_fit = arima_model.fit()
    arima_forecast = arima_fit.forecast()[0]

    # Update ARIMA residuals
    residual = test.iloc[t] - arima_forecast
    history_residuals.append(residual)

    # GARCH Forecast on ARIMA residuals
    garch_model = arch_model(history_residuals, mean='zero', vol='GARCH', p=4, q=2)
    garch_fit = garch_model.fit(disp="off")
    garch_forecast = garch_fit.forecast(horizon=1)
    garch_volatility_forecast = np.sqrt(garch_forecast.variance.values[-1, 0])

    # Combined prediction: ARIMA forecast plus GARCH forecast of residuals
    combined_forecast = arima_forecast + garch_volatility_forecast
    combined_predictions_ResGARCH.append(combined_forecast)

    # Update history
    history_volatility.append(test.iloc[t])

combined_predictions_ResGARCH = pd.Series(combined_predictions_ResGARCH, index=test.index)

# Plot the results
plt.figure(figsize=(6, 4))
plt.plot(realized_volatility[-42:], label='Actual Realized Volatility')
plt.plot(combined_predictions_ResGARCH.index, combined_predictions_ResGARCH, label='ARIMA-GARCH Forecast', linestyle='--')
plt.xticks(rotation=45)
plt.title('Realized Volatility and ARIMA-GARCH Forecast')
plt.legend()
plt.grid(True)
plt.show()

# Error metrics
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape_ResGARCH = mean_absolute_percentage_error(realized_volatility[-21:], combined_predictions_ResGARCH)
mae_ResGARCH = mean_absolute_error(realized_volatility[-21:], combined_predictions_ResGARCH)
rmse_ResGARCH = np.sqrt(mean_squared_error(realized_volatility[-21:], combined_predictions_ResGARCH))
r2_ResGARCH = r2_score(realized_volatility[-21:], combined_predictions_ResGARCH)

# Display metrics
print(f'MAPE: {mape_ResGARCH:.2f}%')
print(f'MAE: {mae_ResGARCH:.2f}')
print(f'RMSE: {rmse_ResGARCH:.2f}')
print(f'R^2: {r2_ResGARCH:.2f}')


# GARCH-ARIMA

In [ ]:
# Split the data into train and test sets
train_volatility = realized_volatility.dropna()[:-21]
test = realized_volatility.dropna()[-21:]

#  GARCH model
garch_model = arch_model(train_volatility, mean = 'zero', vol='EGARCH', p=4, q=2)
garch_fit = garch_model.fit(disp="off")
garch_residuals = garch_fit.resid

 
#  One-step-ahead forecast com ARIMA-GARCH
history_volatility = [x for x in train_volatility]
history_residuals = [x for x in garch_residuals]

combined_predictions_ResARIMA = []

for t in range(len(test)):
    # GARCH Forecast
    garch_model = arch_model(history_volatility, mean = 'zero', vol='EGARCH', p=4, q=2)
    garch_fit = garch_model.fit(disp="off")
    garch_forecast = garch_fit.forecast(horizon=1)
    garch_volatility_forecast = np.sqrt(garch_forecast.variance.values[-1, 0])

    # Update GARCH residuals
    residual = test.iloc[t] - garch_volatility_forecast
    history_residuals.append(residual)


    # ARIMA Forecast on GARCH residuals
    arima_model = ARIMA(history_residuals, order=(4, 0, 4))
    arima_fit = arima_model.fit()
    arima_forecast = arima_fit.forecast()[0]

    # Combined prediction: GARCH forecast plus ARIMA forecast of residuals
    combined_forecast_GARCH_ARIMA = garch_volatility_forecast + arima_forecast
    combined_predictions_ResARIMA.append(combined_forecast_GARCH_ARIMA)

    # Update history
    history_volatility.append(test.iloc[t])

In [ ]:
combined_predictions_ResARIMA = pd.Series(combined_predictions_ResARIMA, index=test.index)

# 10. Plotar um gráfico que mostre a volatilidade realizada nos 21 dias e a predição encontrada pelo GARCH-ARIMA
plt.figure(figsize=(5, 4))
plt.plot(realized_volatility[-42:], label='Actual Realized Volatility')
plt.plot(combined_predictions_ResARIMA.index, combined_predictions_ResARIMA, label='GARCH + Res ARIMA Forecast', linestyle='--')
plt.xticks(rotation=45)  # Rotate x-axis labels by 90 degrees
plt.title('Realized Volatility and GARCH-ARIMA Forecast')
plt.legend()
plt.show()

def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape_GARCH_ResARIMA = mean_absolute_percentage_error(realized_volatility[-21:], combined_predictions_ResARIMA)
# Calcular o MAE
mae_GARCH_ResARIMA = mean_absolute_error(realized_volatility[-21:], combined_predictions_ResARIMA)
# Calcular o RMSE
rmse_GARCH_ResARIMA = np.sqrt(mean_squared_error(realized_volatility[-21:], combined_predictions_ResARIMA))
# Calcular o R^2
r2_GARCH_ResARIMA = r2_score(realized_volatility[-21:], combined_predictions_ResARIMA)

# Exibir as métricas
print(f'MAPE: {mape_GARCH_ResARIMA:.2f}%')
print(f'MAE: {mae_GARCH_ResARIMA*100:.2f}')
print(f'RMSE: {rmse_GARCH_ResARIMA*100:.2f}')
print(f'R^2: {r2_GARCH_ResARIMA:.2f}%')

### WAVELET DECOMPOSITION

In [ ]:
data.head()


In [ ]:
# Select a wavelet function #'db2' (Daubechies wavelet) was suggested
wavelet = 'db2'

# Perform discrete wavelet transform (DWT)
daily_returns= data['Returns']
coeffs = pywt.wavedec(daily_returns, wavelet)

# Coefficients contain both approximation and detail coefficients
approximation_coeffs, detail_coeffs = coeffs[0], coeffs[1:]

### WAVELET RECONSTRUCTION

In [ ]:
# Reconstruct the signal from wavelet coefficients IDWT
reconstructed_returns = pywt.waverec(coeffs, wavelet)

# Align the length of the reconstructed data
reconstructed_returns = reconstructed_returns[:len(daily_returns)]


In [ ]:
# Convert reconstructed_returns to pandas Series
reconstructed_returns_series = pd.Series(reconstructed_returns, index=daily_returns.index)

# Calculate realized volatility (using a rolling window, e.g., 21 days)
window_size = 5
#realized_volatility = daily_returns.rolling(window=window_size).std() * np.sqrt(window_size)

data['Realized_Volatility'] = data['Returns'].rolling(window=window_size).apply(lambda x: np.sqrt(np.sum(x**2)), raw=True)


In [ ]:
realized_volatility= realized_volatility.dropna()
realized_volatility.head(1)

# W-GARCH-ARIMA**

In [ ]:
# 1. Load TSLA data
ticker = 'TSLA'
data = yf.download(ticker, start='2010-07-07', end='2023-02-17')
close_prices = data['Adj Close']

# 2. Calculate daily returns
returns = close_prices.pct_change().dropna()  

# Calculate Daily Realized Volatility
window_size = 5
daily_realized_volatility = returns.rolling(window=window_size).std()



In [ ]:
# Split the data into train and test sets
train_returns = returns.dropna()[:-21]
test = daily_realized_volatility.dropna()[-21:]

In [ ]:

# W-GARCH-ARIMA Model
# Decompose using wavelet transform
wavelet = 'db3' 
coeffs = pywt.wavedec(train_returns, wavelet, level=2, mode='reflect')
approx, detail = coeffs[0], coeffs[1]

# Train GARCH model on the approximate signal (A1)
garch_model = arch_model(approx, mean='zero', vol='EGARCH', p=4, q=2)
garch_fit = garch_model.fit(disp="off")

# Forecast one-step-ahead iteratively
garch_forecast = []
history_approx = list(approx)
for t in range(len(test)):
    garch_fit = arch_model(history_approx, vol='EGARCH', p=4, q=2).fit(disp="off")
    forecast = garch_fit.forecast(horizon=1)
    garch_forecast.append(np.sqrt(forecast.variance.values[-1, 0]))
    history_approx.append(test.iloc[t])

garch_forecast = np.array(garch_forecast)

# Train ARIMA model on the detailed signal (D1)
arima_model = ARIMA(detail, order=(4, 0, 4)).fit()
arima_forecast = arima_model.forecast(steps=len(test))

# Combine forecasts
combined_forecast_W_GARCH_ARIMA = garch_forecast + arima_forecast


# W-ARIMA-GARCH Model
# Decompose time series using wavelet transform
wavelet = 'db4' 
coeffs = pywt.wavedec(train_returns, wavelet, level=3,mode='reflect')
approx, detail = coeffs[0], coeffs[1]

# Train ARIMA model on the approximate signal (A1)
arima_model = ARIMA(approx, order=(4, 0, 4)).fit()
arima_forecast = arima_model.forecast(steps=len(test))

# Train GARCH model on the detailed signal (D1)
garch_model = arch_model(detail, vol='EGARCH', p=4, q=2)
garch_fit = garch_model.fit(disp="off")

# Forecast one-step-ahead iteratively for GARCH
garch_forecast = []
history_detail = list(detail)
for t in range(len(test)):
    garch_fit = arch_model(history_detail, vol='EGARCH', p=4, q=2).fit(disp="off")
    forecast = garch_fit.forecast(horizon=1)
    garch_forecast.append(np.sqrt(forecast.variance.values[-1, 0]))
    history_detail.append(test.iloc[t])

garch_forecast = np.array(garch_forecast)

# Combine forecasts for W-ARIMA-GARCH model
combined_forecast_W_ARIMA_GARCH = arima_forecast + garch_forecast

# Plot the results for W-GARCH-ARIMA Forecast and W-ARIMA-GARCH Forecast
plt.figure(figsize=(5, 4))

# Plot actual realized volatility
plt.plot(daily_realized_volatility[-42:], label='Actual Realized Volatility', color='green')

# Plot the W-GARCH-ARIMA forecast
plt.plot(test.index, combined_forecast_W_GARCH_ARIMA, label='W-GARCH-ARIMA Forecast', linestyle='--', color='blue')

# Plot the W-ARIMA-GARCH forecast
plt.plot(test.index, combined_forecast_W_ARIMA_GARCH, label='W-ARIMA-GARCH Forecast', linestyle='--', color='magenta')

plt.xticks(rotation=45)
plt.title('Real vs W-ARIMA-GARCH and W-GARCH-ARIMA Forecast')
plt.legend()
plt.grid(True)
plt.show()


                                  

In [ ]:

# Calculate metrics for W-GARCH-ARIMA Forecast
mape_w_garch_arima = mean_absolute_percentage_error(daily_realized_volatility[-21:], combined_forecast_W_GARCH_ARIMA)
mae_w_garch_arima = mean_absolute_error(daily_realized_volatility[-21:], combined_forecast_W_GARCH_ARIMA)
rmse_w_garch_arima = np.sqrt(mean_squared_error(daily_realized_volatility[-21:], combined_forecast_W_GARCH_ARIMA))
r2_w_garch_arima = r2_score(daily_realized_volatility[-21:], combined_forecast_W_GARCH_ARIMA)

# Print the results for W-GARCH-ARIMA
print("Metrics for W-GARCH-ARIMA Forecast:")
print(f"MAPE: {mape_w_garch_arima:.4f}")
print(f"MAE: {mae_w_garch_arima:.4f}")
print(f"RMSE: {rmse_w_garch_arima:.4f}")
print(f"R² Score: {r2_w_garch_arima:.4f}")

# Calculate metrics for W-ARIMA-GARCH Forecast
mape_w_arima_garch = mean_absolute_percentage_error(daily_realized_volatility[-21:], combined_forecast_W_ARIMA_GARCH)
mae_w_arima_garch = mean_absolute_error(daily_realized_volatility[-21:], combined_forecast_W_ARIMA_GARCH)
rmse_w_arima_garch = np.sqrt(mean_squared_error(daily_realized_volatility[-21:], combined_forecast_W_ARIMA_GARCH))
r2_w_arima_garch = r2_score(daily_realized_volatility[-21:], combined_forecast_W_ARIMA_GARCH)

# Print the results for W-ARIMA-GARCH
print("\nMetrics for W-ARIMA-GARCH Forecast:")
print(f"MAPE: {mape_w_arima_garch:.4f}")
print(f"MAE: {mae_w_arima_garch:.4f}")
print(f"RMSE: {rmse_w_arima_garch:.4f}")
print(f"R² Score: {r2_w_arima_garch:.4f}")



In [ ]:
returns.head()

In [ ]:
train_returns = returns.dropna()[:-21]
test = train_returns.dropna()[-21:]

In [ ]:

# W-GARCH-ARIMA Model
# Decompose using wavelet transform
wavelet = 'db3' 
coeffs = pywt.wavedec(train_returns, wavelet, level=3, mode='reflect')
approx, detail = coeffs[0], coeffs[1]

# Train GARCH model on the approximate signal (A1)
garch_model = arch_model(approx, mean='zero', vol='EGARCH', p=4, q=2)
garch_fit = garch_model.fit(disp="off")

# Forecast one-step-ahead iteratively
garch_forecast = []
history_approx = list(approx)
for t in range(len(test)):
    garch_fit = arch_model(history_approx, vol='EGARCH', p=4, q=2).fit(disp="off")
    forecast = garch_fit.forecast(horizon=1)
    garch_forecast.append(np.sqrt(forecast.variance.values[-1, 0]))
    history_approx.append(test.iloc[t])

garch_forecast = np.array(garch_forecast)

# Train ARIMA model on the detailed signal (D1)
arima_model = ARIMA(detail, order=(4, 0, 4)).fit()
arima_forecast = arima_model.forecast(steps=len(test))

# Combine forecasts
combined_forecast_W_GARCH_ARIMA = garch_forecast + arima_forecast


# W-ARIMA-GARCH Model
# Decompose time series using wavelet transform
wavelet = 'db4' 
coeffs = pywt.wavedec(train_returns, wavelet, level=3,mode='reflect')
approx, detail = coeffs[0], coeffs[1]

# Train ARIMA model on the approximate signal (A1)
arima_model = ARIMA(approx, order=(4, 0, 4)).fit()
arima_forecast = arima_model.forecast(steps=len(test))

# Train GARCH model on the detailed signal (D1)
garch_model = arch_model(detail, vol='EGARCH', p=4, q=2)
garch_fit = garch_model.fit(disp="off")

# Forecast one-step-ahead iteratively for GARCH
garch_forecast = []
history_detail = list(detail)
for t in range(len(test)):
    garch_fit = arch_model(history_detail, vol='EGARCH', p=4, q=2).fit(disp="off")
    forecast = garch_fit.forecast(horizon=1)
    garch_forecast.append(np.sqrt(forecast.variance.values[-1, 0]))
    history_detail.append(test.iloc[t])

garch_forecast = np.array(garch_forecast)

# Combine forecasts for W-ARIMA-GARCH model
combined_forecast_W_ARIMA_GARCH = arima_forecast + garch_forecast 

# Plot the results for W-GARCH-ARIMA Forecast and W-ARIMA-GARCH Forecast
plt.figure(figsize=(5, 4))

# Plot actual realized volatility
plt.plot(daily_realized_volatility[-42:], label='Actual Realized Volatility', color='green')

# Plot the W-GARCH-ARIMA forecast
plt.plot(test.index, combined_forecast_W_GARCH_ARIMA, label='W-GARCH-ARIMA Forecast', linestyle='--', color='blue')

# Plot the W-ARIMA-GARCH forecast
plt.plot(test.index, combined_forecast_W_ARIMA_GARCH, label='W-ARIMA-GARCH Forecast', linestyle='--', color='magenta')

plt.xticks(rotation=45)
plt.title('Real vs W-ARIMA-GARCH and W-GARCH-ARIMA Forecast')
plt.legend()
plt.grid(True)
plt.show()

